<a href="https://colab.research.google.com/github/ikjot-2605/Interview_Pairing_Automate/blob/improvements/Interview_Pairings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

import pandas as pd

In [ ]:
#interviewer class
class Interviewer:
  def __init__(self, name):
    self.name = name
    self.previouslyInterviewed = []
    self.last_interview_count = 0;
  def addPreviouslyInterviewed(self,interviee_name):
    self.previouslyInterviewed.append(interviee_name)
         
#interviewee class
class Interviewee:
  def __init__(self, name):
    self.name = name
    self.previouslyInterviewedBy = []
  def addPreviouslyInterviewedBy(self,interviewer_name):
    self.previouslyInterviewedBy.append(interviewer_name)      

In [ ]:
sh = gc.open('WEC Mock Interview pairings')
#get the worksheets
worksheet_list = sh.worksheets()
#get the length of list of worksheets
worksheet_list_length = len(worksheet_list)
#get all interviewers and interviewees 
all_interviewees = {};
all_interviewers = {};
#filling out the all_interviewer, all_interviewees dictionaries
for worksheet in worksheet_list:
  #creating dataframe for current sheet
  rows = worksheet.get_all_values()
  pd_val = pd.DataFrame.from_records(rows)
  #iterating through rows of dataframe, and building the respective dicts
  for index, row in pd_val.iterrows():
    if(index>1):
      if(row[0] not in all_interviewees):all_interviewees[row[0]] = (Interviewee(row[0]))
      if(row[1] not in all_interviewers):all_interviewers[row[1]] = (Interviewer(row[1]))
      all_interviewees[row[0]].addPreviouslyInterviewedBy(row[1])
      all_interviewers[row[1]].addPreviouslyInterviewed(row[0]) 
    
   

In [ ]:
#set the previous 2 interview takers
rows = worksheet_list[len(worksheet_list)-1].get_all_values()
pd_val = pd.DataFrame.from_records(rows)
for index,row in pd_val.iterrows():
  if(index>1):
    all_interviewers[row[1]].last_interview_count=all_interviewers[row[1]].last_interview_count+1;
hash = sorted(all_interviewers.values(), key=lambda interviewer: interviewer.last_interview_count)  
newhash = {}
for interviewer in hash:
  newhash[interviewer.name] = interviewer
all_interviewers = newhash  

In [ ]:
for interviewer in all_interviewers:
  all_interviewers[interviewer].remaining_interviews = 2 - all_interviewers[interviewer].last_interview_count;  

2
2
2
2
2
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
interview_pairings = []
for interviewee in all_interviewees:
  interview_pairings.append([all_interviewees[interviewee].name])
i=0
for interviewer in all_interviewers:
  interview_pairings[i].append(all_interviewers[interviewer].name)
  i=i+1
print(interview_pairings)  